In [ ]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from pathlib import Path
from urllib.parse import urljoin

# --- CONFIGURACIÓN Y UTILIDADES ---
BASE_URL = "https://books.toscrape.com/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def safe_int(text, default=None):
    try:
        return int(text)
    except (ValueError, TypeError):
        return default


def safe_float(text, default=None):
    try:
        return float(text)
    except (ValueError, TypeError):
        return default
    

def rating_to_int(rating_class):
    # Mapeo de palabras de estrellas a números
    ratings = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    for word, value in ratings.items():
        if word in rating_class:
            return value
    return 0

def scrape_books():
    datos_libros = []
    
    # 1. Obtener categorías
    response = requests.get(BASE_URL, headers=HEADERS, timeout=1)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    
    # El sidebar contiene las categorías (omitimos el primero que es "Books")
    categories_tags = soup.find("ul", class_="nav-list").find("ul").find_all("a")
    
    for cat_tag in categories_tags:
        cat_name = cat_tag.get_text(strip=True)
        cat_url = urljoin(BASE_URL, cat_tag['href'])
        
        print(f"Scrapeando categoría: {cat_name}...")
        
        # 2. Navegar por las páginas de la categoría
        current_page_url = cat_url
        while current_page_url:
            cat_res = requests.get(current_page_url)
            cat_soup = BeautifulSoup(cat_res.text, "html.parser")
            
            books_in_page = cat_soup.find_all("article", class_="product_pod")
            
            for book in books_in_page:
                # Obtener URL del detalle del libro
                relative_book_url = book.find("h3").find("a")["href"]
                book_url = urljoin(current_page_url, relative_book_url)
                
                # ENTRAR AL DETALLE DEL LIBRO
                book_res = requests.get(book_url)
                book_soup = BeautifulSoup(book_res.text, "html.parser")
                
                # Nombre
                title = book_soup.find("h1").get_text(strip=True)
                
                # Precio (Convertir a float)
                price_text = book_soup.find("p", class_="price_color").get_text(strip=True)
                price_float = safe_float(re.sub(r'[^\d.]', '', price_text))
                
                # Stock (Convertir a int)
                stock_text = book_soup.find("p", class_="instock availability").get_text(strip=True)
                stock_int = safe_int(re.search(r'\d+', stock_text).group())
                
                # Estrellas
                rating_class = book_soup.find("p", class_="star-rating")["class"]
                rating_int = rating_to_int(rating_class)

                # Imagen (Descarga y Ruta)
                img_relative_url = book_soup.find("div", class_="item active").find("img")["src"]
                img_url = urljoin(BASE_URL, img_relative_url)
                
                # Guardar en lista
                datos_libros.append({
                    "Nombre": title,
                    "Categoría": cat_name,
                    "Precio": price_float,
                    "Stock": stock_int,
                    "Estrellas": rating_int,
                    "Ruta_Imagen": str(img_url)
                })
            
            # Ver si hay página siguiente en la categoría
            next_button = cat_soup.find("li", class_="next")
            if next_button:
                next_url = next_button.find("a")["href"]
                current_page_url = urljoin(current_page_url, next_url)
            else:
                current_page_url = None
                
    return datos_libros

# --- EJECUCIÓN ---

if __name__ == "__main__":
    lista_final = scrape_books()
    
    # Crear DataFrame
    df = pd.DataFrame(lista_final)
    
    # Mostrar resultados
    print("\nScraping completado.")
    print(f"Total de libros recolectados: {len(df)}")
    
    # Mostrar las primeras filas
    display(df.head())

Scrapeando categoría: Travel...
Scrapeando categoría: Mystery...
Scrapeando categoría: Historical Fiction...
Scrapeando categoría: Sequential Art...
Scrapeando categoría: Classics...
Scrapeando categoría: Philosophy...
Scrapeando categoría: Romance...
Scrapeando categoría: Womens Fiction...
Scrapeando categoría: Fiction...
Scrapeando categoría: Childrens...
Scrapeando categoría: Religion...
Scrapeando categoría: Nonfiction...
Scrapeando categoría: Music...
Scrapeando categoría: Default...
Scrapeando categoría: Science Fiction...
Scrapeando categoría: Sports and Games...
Scrapeando categoría: Add a comment...
Scrapeando categoría: Fantasy...
Scrapeando categoría: New Adult...
Scrapeando categoría: Young Adult...
Scrapeando categoría: Science...
Scrapeando categoría: Poetry...
Scrapeando categoría: Paranormal...
Scrapeando categoría: Art...
Scrapeando categoría: Psychology...
Scrapeando categoría: Autobiography...
Scrapeando categoría: Parenting...
Scrapeando categoría: Adult Fiction...


,Nombre,Categoría,Precio,Stock,Estrellas,Ruta_Imagen
0,It's Only the Himalayas,Travel,45.17,19,2,https://books.toscrape.com/media/cache/6d/41/6...
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,Travel,49.43,15,4,https://books.toscrape.com/media/cache/fe/8a/f...
2,See America: A Celebration of Our National Par...,Travel,48.87,14,3,https://books.toscrape.com/media/cache/c7/1a/c...
3,Vagabonding: An Uncommon Guide to the Art of L...,Travel,36.94,8,2,https://books.toscrape.com/media/cache/ca/30/c...
4,Under the Tuscan Sun,Travel,37.33,7,3,https://books.toscrape.com/media/cache/45/21/4...
